In [1]:
from coinapy.fetcher import get_exchange_symbols, get_ohlcv

True


/Users/doyeon/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


- BTC, ETH, SOL, ADA, DOGE
- TRON, XRP

In [43]:
import os 
PERIODS = ["5SEC"]  # 5초, 1분 단위 데이터
# target = "BTC" 
# target = "ETH"
target = "XRP"
# exchanges = ["BINANCE", "UPBIT"]
# exchanges = ["COINONE", "BITHUMB"]
exchanges = ["BINANCE", "COINBASE", "OKEX", "CRYPTOCOM"] # "BYBIT", 
# for o in PERIODS:
#     os.makedirs(f"data/{o}", exist_ok=True)
#     for oo in exchanges:
#         os.makedirs(f"data/{o}/{oo}", exist_ok=True)


In [44]:
import json
with open('symbols.json', 'r') as f:
    d = json.loads(f.read())

In [36]:
x = set()
for dd in d:
    # x.add(
    #     dd["exchange_id"]
    # )
    if dd["exchange_id"] == "BYBIT":
        print(dd["symbol_id"])


BYBIT_PERP_BTC_USDT
BYBIT_PERP_ETH_USDT
BYBIT_PERP_LTC_USDT
BYBIT_PERP_LINK_USDT
BYBIT_PERP_XTZ_USDT
BYBIT_PERP_BCH_USDT
BYBIT_PERP_ADA_USDT
BYBIT_PERP_DOT_USDT
BYBIT_PERP_UNI_USDT
BYBIT_PERP_XRP_USDT
BYBIT_PERP_AAVE_USDT
BYBIT_PERP_SUSHI_USDT
BYBIT_PERP_XEM_USDT
BYBIT_PERP_DOGE_USDT
BYBIT_PERP_SOL_USDT
BYBIT_PERP_BNB_USDT
BYBIT_PERP_ETC_USDT
BYBIT_PERP_FIL_USDT
BYBIT_PERP_EOS_USDT
BYBIT_PERP_XLM_USDT
BYBIT_PERP_TRX_USDT
BYBIT_PERP_AXS_USDT
BYBIT_PERP_COMP_USDT
BYBIT_PERP_THETA_USDT
BYBIT_PERP_AVAX_USDT
BYBIT_PERP_ICP_USDT
BYBIT_PERP_ALGO_USDT
BYBIT_PERP_NEAR_USDT
BYBIT_PERP_ATOM_USDT
BYBIT_PERP_DYDX_USDT
BYBIT_PERP_CHZ_USDT
BYBIT_PERP_KSM_USDT
BYBIT_PERP_OMG_USDT
BYBIT_PERP_DASH_USDT
BYBIT_PERP_IOST_USDT
BYBIT_PERP_SHIB1000_USDT
BYBIT_PERP_HBAR_USDT
BYBIT_PERP_CRV_USDT
BYBIT_PERP_VET_USDT
BYBIT_PERP_ONE_USDT
BYBIT_PERP_C98_USDT
BYBIT_PERP_ALICE_USDT
BYBIT_PERP_CELR_USDT
BYBIT_PERP_SAND_USDT
BYBIT_PERP_ENJ_USDT
BYBIT_PERP_MANA_USDT
BYBIT_PERP_GALA_USDT
BYBIT_PERP_WOO_USDT
BYBIT_PERP_LR

In [45]:
symbols = get_exchange_symbols(target, exchanges)
if not symbols:
    print("No symbols")

In [38]:
symbols

{'BINANCE': 'BINANCE_SPOT_ETH_USDT',
 'OKEX': 'OKEX_SPOT_ETH_USDT',
 'COINBASE': 'COINBASE_SPOT_ETH_USDT',
 'CRYPTOCOM': 'CRYPTOCOM_SPOT_ETH_USDT'}

In [46]:
from datetime import datetime, timezone, timedelta

def kst_to_utc(iso_time_str: str):
    # Convert ISO string to datetime object (set KST timezone)
    kst_time = datetime.fromisoformat(iso_time_str).astimezone(timezone(timedelta(hours=9)))
    
    # Convert to UTC
    utc_time = kst_time.astimezone(timezone.utc)

    # Ensure microseconds are formatted to 7 digits and replace timezone with 'Z'
    return utc_time.strftime("%Y-%m-%dT%H:%M:%S.%f")[:26] + "0Z"

# Example usage
# print(kst_to_utc("2025-03-05T21:15:00.0100000+09:00"))
# Output: 2025-03-05T12:15:00.0100000Z


In [7]:
from coinapy.util import kst_to_utc
# Example usage
print(kst_to_utc("2025-03-05T21:15:00.010000+09:00"))
# Output: 2025-03-05T12:15:00.0100000Z


2025-03-05T12:15:00.0100000Z


In [8]:
# from datetime import datetime, timedelta

# def count_intervals(start: str, end: str, period: str) -> int:

#     # ISO 8601 시간 문자열을 datetime 객체로 변환
#     start_time = datetime.fromisoformat(start)
#     end_time = datetime.fromisoformat(end)

#     value = int(period[:-1])  # 숫자 부분 추출
    
#     if period[-3:] == 'OUR': # hour
#         delta = timedelta(hours=value)
#     elif period[-3:] == 'MIN':
#         delta = timedelta(minutes=value)
#     elif period[-3:] == 'SEC':
#         delta = timedelta(seconds=value)
#     else:
#         raise ValueError("Invalid period format. Use 'h' (hours), 'm' (minutes), or 's' (seconds).")

#     # 총 간격 개수 계산
#     total_intervals = (end_time - start_time) // delta

#     return total_intervals

# # ✅ 사용 예시
# # print(count_intervals("2025-03-05T00:00:00.0000000Z", "2025-03-05T12:00:00.0000000Z", "1h"))  # 출력: 12
# print(count_intervals("2025-03-05T12:00:00.000000Z", "2025-03-05T12:30:00.000000Z", "1MIN"))  # 출력: 3
# print(count_intervals("2025-03-05T12:00:00.0000000Z", "2025-03-05T12:01:00.0000000Z", "5SEC"))  # 출력: 4


In [9]:
target

'BTC'

In [30]:
symbols

{'BINANCE': 'BINANCE_SPOT_BTC_USDT',
 'OKEX': 'OKEX_SPOT_BTC_USDT',
 'COINBASE': 'COINBASE_SPOT_BTC_USDT',
 'CRYPTOCOM': 'CRYPTOCOM_SPOT_BTC_USDT'}

In [ ]:
# data = []
# limit = 1000
# times = "2024-12-01T22:00:00.010000+09:00"
# time_start = kst_to_utc(times)
# time_end = datetime.strptime(times, 
#                             "%Y-%m-%dT%H:%M:%S.%f%z"
#                     ) + timedelta(seconds=limit * 5) # 5sec  
# time_end = time_end.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "00000Z"

# time_start, time_end

('2024-12-01T13:00:00.0100000Z', '2024-12-01T23:23:20.01000000Z')

In [42]:
data = []
time_start = kst_to_utc("2024-12-01T22:00:00.010000+09:00")
time_end = kst_to_utc("2024-12-05T22:00:00.010000+09:00")
limit = 100000

for exchange, symbol_id in symbols.items():
    if not symbol_id:
        print(f"⚠ Cannot find {target} in {exchange}")
        continue
    
    print(f"📌 {exchange} {target} symbol_id: {symbol_id}")
    sym_id =  "_".join(symbol_id.split("_")[1:])
    for period in PERIODS:
        os.makedirs(f"data/{period}/{exchange}/{sym_id}", exist_ok=True)
        filename = f"data/{period}/{exchange}/{sym_id}/{time_start}_{time_end}.csv"
        print(f"📝 {filename}")
        if os.path.exists(filename):
            print(f"📂 {symbol_id} already exists")
            continue

        data = get_ohlcv(
            symbol_id, period, time_start, time_end, limit=limit)
        if not data.empty:
            data.to_csv(filename, index=False)
        else:
            print(f"❌ No data for {exchange} {symbol_id} {period}")

📌 BINANCE ETH symbol_id: BINANCE_SPOT_ETH_USDT
📝 data/5SEC/BINANCE/SPOT_ETH_USDT/2024-12-02T13:00:00.0100000Z_2024-12-03T13:00:00.0100000Z.csv
✅ BINANCE_SPOT_ETH_USDT - 5SEC downloaded! (15762 rows)
📌 OKEX ETH symbol_id: OKEX_SPOT_ETH_USDT
📝 data/5SEC/OKEX/SPOT_ETH_USDT/2024-12-02T13:00:00.0100000Z_2024-12-03T13:00:00.0100000Z.csv
✅ OKEX_SPOT_ETH_USDT - 5SEC downloaded! (15322 rows)
📌 COINBASE ETH symbol_id: COINBASE_SPOT_ETH_USDT
📝 data/5SEC/COINBASE/SPOT_ETH_USDT/2024-12-02T13:00:00.0100000Z_2024-12-03T13:00:00.0100000Z.csv
✅ COINBASE_SPOT_ETH_USDT - 5SEC downloaded! (5973 rows)
📌 CRYPTOCOM ETH symbol_id: CRYPTOCOM_SPOT_ETH_USDT
📝 data/5SEC/CRYPTOCOM/SPOT_ETH_USDT/2024-12-02T13:00:00.0100000Z_2024-12-03T13:00:00.0100000Z.csv
✅ CRYPTOCOM_SPOT_ETH_USDT - 5SEC downloaded! (13800 rows)


In [ ]:
import os

os.makedirs("data", exist_ok=True)

for d in data:
    if d.empty:
        continue

    exchange = d[0]["exchange"]
    symbol = d[0]["symbol"]
    period = d[0]["period"]

KeyError: 0